## Task 1. Run a batch data to get the loss value

ignore the bert mask

记录问题openfold/feats.py/torsion_angles_to_frames/default_4x4 = rrgdf[aatype, ...]报错

一个经典的问题，如何用一个tensor对另一个tensor切片----比较灵活的常量高维embedding

我定义的dataset : 0-19 + X + padding 一共22个token，但是只定义了21种frame constant

target_tensor[query_tensor] 进行检索时，实际是用query_sequence的每一个值去检索target_tensor的dim0 对应的向量

返回的shape是 (*target_tensor.shape,*query_tensor.shape[:1])

必须要求query_tensor是bool，Long或binary且query_tensor的每一个值都小于target_tensor dim0的数量

类似于embedding index的操作

In [1]:
import torch
import torch.nn as nn
import json,time
import customize_data
import utils
import numpy as np
import esm.esmfold.v1.esmfold as ESM
import modules 
import os
from openfold.utils.rigid_utils import Rigid
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/opt/conda/envs/esmfold/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset
data_file = "data/chain_set.jsonl"
dataset = customize_data.StructureDataset(data_file)
print(f"bad seq:{dataset.discard['bad_chars']}\ntoo long:{dataset.discard['too_long']}")

155c.A {'X'} XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRKIASEEGFKYGEGILEVAEKNPDLTWTEANLIEYVTDPKPLVKKMTDDKGAKTKMTFKMGKNQADVVAFLAQDDPDAXXXXXXXXXXXXX
1a0r.B {'X'} XSELDQLRQEAEQLKNQIRDARKACADATLSQITNNIDPVGRIQMRTRRTLRGHLAKIYAMHWGTDSRLLLSASQDGKLIIWDSYTTNKVHAIPLRSSWVMTCAYAPSGNYVACGGLDNICSIYNLKTREGNVRVSRELAGHTGYLSCCRFLDDNQIVTSSGDTTCALWDIETGQQTTTFTGHTGDVMSLSLAPDTRLFVSGACDASAKLWDVREGMCRQTFTGHESDINAICFFPNGNAFATGSDDATCRLFDLRADQELMTYSHDNIICGITSVSFSKSGRLLLAGYDDFNCNVWDALKADRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLKIWN
1a48.A {'X'} XSITKTELDGILPLVARGKVRDIYEVDAGTLLFVATDRISAYDVIMENSIPEKGILLTKLSEFWFKFLSNDVRNHLVDIAPGKTIFDYLPAKLSEPKYKTQLEDRSLLVHKHKLIPLEVIVRGYITGSAWKEYVKTGTVHGLKQPQGLKESQEFPEPIFTPSTKAEQGEHDENISPAQAAELVGEDLSRRVAELAVKLYSKCKDYAKEKGIIIADTKFEFGIDEKTNEIILVDEVLTPDSSRFWNGASYKVGESQDSYDKQFLRDWLTANKLNGVNGVKMPQDIVDRTRAKYIEAYETLTGSKWSH
1auy.B {'X'} XMEIDKELAPQDRTVTVATVLPAVPGPSPLTIKQPFQSEVLFAGTKDAEASLTIANIDSVSTLTTFYRHASLESLWVTIHPTLQAPTFPTTVGVCWVPAQSPVTPAQITKTYGGQIFCIGGAIQTLSPLIVKCPLEMMQPRVKDSIQYLDSPKLLISITAQPTA

In [3]:
# dataloader
dataLoader = customize_data.StructureDataloader(dataset,4,num_workers=1)
batch_example = next(iter(dataLoader))

In [4]:
# Reading the data file and initialize the esm-inpainting class
model_path = "/root/.cache/torch/hub/checkpoints/esmfold_3B_v1.pt"
model_data = torch.load(str(model_path), map_location="cuda:0") #读取一个pickle文件为一个dict
cfg = model_data["cfg"]["model"]
model = modules.esm_inpaint(cfg) # make an instance
model_state = model_data["model"]
model.esmfold.load_state_dict(model_state, strict=False)
model.esmfold.set_chunk_size(128)

In [5]:
model = model.to("cuda")
# 11735MiB / 32510MiB

In [6]:
utils.recur_print(batch_example)

{'coord': 'torch.Size([4, 378, 4, 3])_torch.float32',
 'seq': 'torch.Size([4, 378])_torch.int64',
 'mask_coord': 'torch.Size([4, 378, 4, 3])_torch.float32',
 'mask_seq': 'torch.Size([4, 378])_torch.int64',
 'bert_mask_fraction': 'torch.Size([4, 1])_torch.float32',
 'bert_mask': 'torch.Size([4, 378])_torch.bool',
 'padding_mask': 'torch.Size([4, 378])_torch.float32'}

In [7]:
for key in batch_example.keys():
    batch_example[key] = batch_example[key].to("cuda")

In [8]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
output = model(coord=batch_example['mask_coord'],mask=(batch_example['padding_mask']).to(torch.float32),S=batch_example['mask_seq'])

torch.float32
coord cuda:0
 mask cuda:0
 mask_frame cuda:0
embedding cuda:0
 bb_frame cuda:0
torch.Size([4, 378, 378, 128])
Recycling:0-----------
Folding Trunk
Structure Module
Recycling Output
Recycling:1-----------
Folding Trunk
Structure Module
Recycling Output
Recycling:2-----------
Folding Trunk
Structure Module
Recycling Output
Recycling:3-----------
Folding Trunk
Structure Module
Recycling Output


/root/ESM-Inpainting/esm_inpaint/modules.py:88: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output_seq = F.log_softmax(structure['lm_logits'])


## Seq Loss

cross entropy loss

In [9]:
seq = batch_example['seq']
log_pred = output['log_softmax_aa']
padding_mask = batch_example['padding_mask']
_, loss_av_smoothed = utils.loss_nll(seq, log_pred, padding_mask)
loss_av_smoothed

tensor(8.7960, device='cuda:0')

## Backbone Frame Loss

In [21]:
B,L = output['positions'].shape[:2]
pred_position = output['positions'].reshape(B, -1, 3)
target_position = batch_example['coord'][:,:,:3,:].reshape(B, -1 ,3)
position_mask = torch.ones_like(target_position[...,0])
# utils.compute_fape(Rigid.from_tensor_7(output['pred_frames']),output['target_frames'],batch_example['padding_mask'],pred_position,target_position,position_mask,1)

utils.compute_fape(output['pred_frames'],output['target_frames'],batch_example['padding_mask'],pred_position,target_position,position_mask,10.0)

tensor([20.4501, 46.4380, 26.0358, 91.8670], device='cuda:0')